In [8]:
import datasets
from datasets import load_dataset

dataset = load_dataset("/mnt/petrelfs/songmingyang/songmingyang/data/reasoning/gsm8k","main")


In [10]:
dataset["test"]

Dataset({
    features: ['question', 'answer'],
    num_rows: 1319
})

In [11]:
dataset["test"][0]

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}

In [1]:
from r1_p.utils.utils import *

res_file = "/mnt/petrelfs/songmingyang/code/reasoning/others/stare_open/vllm_inference/qwq/scripts/qwq_gsm8k.jsonl"
res = process_jsonl(res_file)
len(res)

1319

In [2]:
output_lengths = []
input_lengths = []
for item in res:
    output_lengths.append(item["output_length"])
    input_lengths.append(item["input_length"])

output_length = sum(output_lengths) / len(output_lengths)
input_length = sum(input_lengths) / len(input_lengths)
print(f"Output length: {output_length}, Input length: {input_length}")

Output length: 1579.8294162244124, Input length: 75.03108415466262


In [3]:
output_lengths = []
input_lengths = []
for item in res:
    output_lengths.append(item["output_length"])
    input_lengths.append(len(item["question"].split()))

output_length = sum(output_lengths) / len(output_lengths)
input_length = sum(input_lengths) / len(input_lengths)
print(f"Output length: {output_length}, Input length: {input_length}")

Output length: 1579.8294162244124, Input length: 46.25094768764215


In [19]:
import re
from typing import Optional, Union
from collections import Counter
from math_verify import verify,parse

def retrieve_answer(output) -> Optional[str]:
    '''
    output should be a world_model.GSM8kState if being a list
    '''
    try:
        res = parse(output)[0]
    except:
        res = output.split("\\boxed{")[-1].split("}")[0]
    
    return str(res)


def retrieve_answer_from_dataset(answer: str) -> str:
    return re.match(r'[\S\s]*#### (.*)$', answer)[1]


def judge_answer(output: Optional[str], answer: str) -> bool:
    if output is None:
        return False
    try:
        res = verify(output, answer)
    except:
        return False
    if not res:
        try:
            newres = float(output) == float(answer)
        except:
            newres = False
        res = newres
    if not res:
        print(f"Output: {output}, Answer: {answer}")
    return res

In [20]:
output_lengths = []
input_lengths = []
acc_list = []
for item in res:
    output_lengths.append(item["output_length"])
    input_lengths.append(item["input_length"])
    answer = retrieve_answer_from_dataset(item["answer"])
    output = retrieve_answer(item["response"])
    acc_list.append(judge_answer(output, answer))

output_length = sum(output_lengths) / len(output_lengths)
input_length = sum(input_lengths) / len(input_lengths)
acc = sum(acc_list) / len(acc_list)
print(f"Output length: {output_length}, Input length: {input_length}, Acc: {acc}")
# print(f"Output length: {output_length}, Input length: {input_length}")

Output: Eq(-30 + 100, 70), Answer: 70
Output: 400/11, Answer: 36
Output: 765, Answer: 175
Output: 99077, Answer: 95200
Output: 20*(1/100), Answer: 20
Output: 2125, Answer: 2,125
Output: 100*(1/100), Answer: 25
Output: 114200, Answer: 114,200
Output: {30, E*a*a*f*h*i*o*o*o*p*r*s*s*s*t*t*w}, Answer: 10
Output: 276000, Answer: 276,000
Output: 3, Answer: 6
Output: 5600, Answer: 5,600
Output: 4, Answer: 3
Output: 3700, Answer: 3200
Output: 6, Answer: 4
Output: 10*(1/100), Answer: 10
Output: 10, Answer: 18
Output: 760.420000000000, Answer: 803
Output: 135, Answer: 81
Output: 240, Answer: 200
Output: 240, Answer: 150
Output: 35*(1/100), Answer: 35
Output: 10*(1/100), Answer: 10
Output: 1600, Answer: 1,600
Output: 522, Answer: 525
Output: 288, Answer: 160
Output: 900, Answer: 450
Output: 16*E*b*f*h*l*o*o*o*p*r*s*s*t*t*u, Answer: 1
Output: 65960, Answer: 65,960
Output: 1450000, Answer: 1,450,000
Output: Eq(1200 + 1600 + 2000, 4800), Answer: 4800
Output: 43500, Answer: 43,500
Output: 10800, Answ

In [16]:
verify("6","6.0000000")

False

In [9]:
text = """
Okay, so Henry went on a 60-mile bike trip and made two stops. The problem is asking how many miles he traveled between his first and second stops. Let me break this down step by step.

First, the total trip is 60 miles. He stopped the first time after 20 miles. That part seems straightforward. So from the start to the first stop, he did 20 miles. Now, the second stop was 15 miles before the end of the trip. Hmm, I need to figure out where that second stop is located in terms of miles from the start.

If the entire trip is 60 miles, then 15 miles before the end would mean he had 15 miles left to go after the second stop. So, to find the distance from the start to the second stop, I can subtract those 15 miles from the total 60. Let me write that out: 60 miles total - 15 miles remaining = 45 miles from the start. So the second stop was at the 45-mile mark.

Now, the question is the distance between the first and second stops. The first stop was at 20 miles, the second at 45 miles. So subtracting the two should give the distance between them: 45 miles - 20 miles = 25 miles. 

Wait a second, let me double-check. If he stopped after 20 miles, then continued riding and stopped again 15 miles before the end, which we figured out is at 45 miles. So from 20 to 45, that's 25 miles. Then from the second stop, he had 15 miles left to finish the trip. Adding those up: 20 + 25 + 15 should equal the total 60 miles. Let me confirm: 20 +25 is 45, plus 15 is 60. Yep, that adds up. 

I think that makes sense. So the answer should be 25 miles between the first and second stops. I don't see any other factors here. The problem doesn't mention any detours or extra miles, so assuming he went straight from the first stop to the second stop and then to the end. Yeah, that seems right. So the answer is 25.
</think>

Henry's bike trip was 60 miles in total. He made his first stop after riding 20 miles. The second stop occurred 15 miles before the end of the trip, which means he had 15 miles remaining after the second stop. To find the distance from the start to the second stop, subtract the remaining 15 miles from the total distance:  
\[ 60 - 15 = 45 \text{ miles, Answer: 25
Output: Okay, so Grandma Jones made 5 apple pies, right? Each pie is cut into 8 pieces. The question is asking how many pieces the guests took if there were 14 pieces left at the end. Hmm, let me think step by step.

First, I need to figure out the total number of pie pieces she started with. Since each pie is divided into 8 pieces and there are 5 pies, I can multiply 5 by 8. Let me write that down: 5 pies × 8 pieces per pie. 5 times 8 is 40. So, there were 40 pieces in total at the beginning.

Now, the problem says that after the guests took their pieces, there were 14 left. To find out how many were taken, I guess I need to subtract the remaining pieces from the original total. That makes sense because the total minus what's left should give what was taken. So, 40 total pieces minus 14 remaining equals the number of pieces taken. Let me do the subtraction: 40 - 14. Hmm, 40 minus 10 is 30, and then minus 4 more would be 26. Wait, is that right? Let me check again. 40 minus 14. If I take away 10 from 40, that's 30, then subtract the remaining 4, so 30 -4 =26. Yeah, that seems correct. 

But let me make sure I didn't make a mistake in the initial calculation. So 5 pies each cut into 8 slices: 5×8 is indeed 40. Then subtract the 14 left, so 40-14=26. Hmm, that seems straightforward. 

Wait, is there another way to approach this? Maybe adding up the pieces taken instead? Let's see. If each pie was cut into 8 pieces, and there are 5 pies, so total is 40. The remaining is 14, so the guests must have taken 40-14. Yeah, that's the same as before. 

Alternatively, maybe breaking it down: if there were 14 pieces left, how many pies is that? Since each pie is 8 pieces, 14 pieces would be 1 and 3/4 pies left? But maybe that's complicating it. The question just asks for the number of pieces taken, so the straightforward subtraction should be enough. 

I think I might be overcomplicating. Let me just verify the arithmetic again. 40 minus 14. 40 minus 10 is 30, minus 4 is 26. Alternatively, 14 plus 26 equals 40. Yes, that checks out. So the answer should be 26 pieces taken by the guests. 

I don't see any other factors here. The problem doesn't mention any pies being damaged or thrown away, just that the guests took and ate their pieces, leaving 14. So the calculation is straightforward. Total minus remaining equals taken. 40-14=26. Yeah, that's solid. I think that's the answer.
</think>

Grandma Jones started with 5 pies, each cut into 8 pieces, resulting in a total of \(5 \times 8 = 40\) pieces. After the event, 14 pieces remained. To find the number of pieces taken by the guests, subtract the remaining pieces from the total: 

\[
40 - 14 = 26
\]

**Answer:** 26 pieces were taken by the guests.
"""


In [10]:
from math_verify import parse
a = parse(text)

In [11]:
a

[26, '26']